In [ ]:
#### import global modules
import os
import sys
import pandas as pd
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])

In [ ]:
pd.options.display.max_rows = 100

In [ ]:
Customer_Detail_Query= '''

SELECT a.*  FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` as a
left join `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_terminated_details` as b
on a.customer_id=b.customer_id WHERE b.customer_id IS NULL
and a.last_updt_ts=(SELECT MAX(st1.last_updt_ts) FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` AS st1) 


'''

In [ ]:
Network_Query='''


with Customer_details as
 (
SELECT a.customer_id,
  FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` as a
left join `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_terminated_details` as b
on a.customer_id=b.customer_id WHERE b.customer_id IS NULL
and a.last_updt_ts=(SELECT MAX(st1.last_updt_ts) FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` AS st1) 
),


-- OLDER METHOD FOR MAX RECORD
--  signal_info_interim as (
--    select customer_id, network_desc, history_date_utc
--          ,row_number() over (partition by customer_id order by history_date_utc desc) as rn
--    from `cio-datahub-enterprise-pr-183a.src_adc.bq_signal_rating_history`
-- ),

-- OLDER METHOD FOR MAX RECORD

-- signal_info as
-- (SELECT * FROM signal_info_interim where rn=1),




-- NEW METHOD FOR MAX RECORD
signal_info_final as
(select customer_id,network_id, network_desc,signaling_rating_id,signaling_rating_name,sinaling_rating_logic, history_date_utc as signal_rating_date from `cio-datahub-enterprise-pr-183a.src_adc.bq_signal_rating_history` 
QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY history_date_utc DESC) = 1)
-- ,


select cd.* ,sig.network_id,sig.network_desc,sig.signaling_rating_id,sig.signaling_rating_name,sig.sinaling_rating_logic, sig.signal_rating_date from Customer_details cd 
left join signal_info_final sig 
on  cd.customer_id=sig.customer_id
order by cd.customer_id




'''

In [ ]:
Trouble_condition_Query='''

select * from `cio-datahub-enterprise-pr-183a.src_adc.bq_troublecondition_data`

'''

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Customer_Network_details_DF= extract_bq_data(bq_client,sql=Network_Query)

In [ ]:
Customer_Network_details_DF.info()

In [ ]:
Customer_Network_details_DF.isnull().sum() * 100 / len(Customer_Network_details_DF)

In [ ]:
Customer_Network_details_DF.head()

In [ ]:
Customer_Network_details_DF['network_desc'].fillna('Missing_network_information',inplace=True)

In [ ]:
Customer_Network_details_DF.value_counts('network_desc',normalize=True)*100

In [ ]:
job_config_Network = bigquery.job.LoadJobConfig()
job_config_Network.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Network_BQ_table = 'ADC.Customer_Network_details'

job_device_type= bq_client.load_table_from_dataframe(Customer_Network_details_DF, Network_BQ_table,job_config=job_config_Network)

In [ ]:
Customer_Details_DF= extract_bq_data(bq_client,sql=Customer_Detail_Query)

In [ ]:
Customer_Details_DF.info()

In [ ]:
Customer_Details_DF['dealer_name'].value_counts()

In [ ]:
Customer_Details_DF_NonADT=Customer_Details_DF[(Customer_Details_DF['dealer_name']=='TELUS Communications Inc.') & (Customer_Details_DF['customer_type_name']=='Customer') ]

In [ ]:
Customer_Details_DF_NonADT.shape

In [ ]:
Customer_Details_DF_NonADT.to_csv('NON_ADT_Customer_BQ_ADC.csv',index=False)

In [ ]:
Trouble_condition_DF= extract_bq_data(bq_client,sql=Trouble_condition_Query)

In [ ]:
Trouble_condition_DF.head()

In [ ]:
print(Trouble_condition_DF['start_date_utc'].min(),'and',Trouble_condition_DF['start_date_utc'].max())

In [ ]:
Trouble_condition_DF['trouble_condition_group_desc'].value_counts(normalize=True)*100

In [ ]:
Trouble_condition_DF['trouble_condition_name'].value_counts(normalize=True)*100

In [ ]:
Trouble_condition_DF['trouble_condition_start_ind'].value_counts(normalize=True)*100

In [ ]:
Trouble_condition_DF['trouble_condition_closed_ind'].value_counts(normalize=True)*100

In [ ]:
Trouble_condition_DF[Trouble_condition_DF['trouble_condition_closed_ind']=='N']['start_date_utc'].min()

In [ ]:
Trouble_condition_DF[Trouble_condition_DF['trouble_condition_start_ind']=='N']['start_date_utc'].max()

In [ ]:
Trouble_condition_DF[Trouble_condition_DF['start_date_utc']=='2011-07-27 10:27:55+0000']

In [ ]:
Trouble_condition_DF['dealer_name'].value_counts(normalize=True)*100

In [ ]:
Trouble_condition_DF['Day_Diff']=Trouble_condition_DF['end_date_utc'] - Trouble_condition_DF['start_date_utc']

In [ ]:
Trouble_condition_DF.info()

In [ ]:
Trouble_condition_DF['Day_Diff'].min()

In [ ]:
Trouble_condition_DF['Day_Diff'].max()

In [ ]:
Trouble_condition_DF['Day_Diff'].median()

In [ ]:
Trouble_condition_DF['Day_Diff'].mean()

In [ ]:
Trouble_condition_DF.groupby('trouble_condition_group_desc')['Day_Diff'].median()

In [ ]:
Trouble_condition_DF.groupby('trouble_condition_group_desc')['Day_Diff'].mean()

In [ ]:
Current_device_Query='''

select * from `cio-datahub-enterprise-dv-e8ff.src_adc.bq_current_device`

'''

In [ ]:
Current_Device_DF= extract_bq_data(bq_client,sql=Current_device_Query)

In [ ]:
Current_Device_DF.info()

In [ ]:
Current_Device_DF.head()

In [ ]:
Current_Device_DF['partition_device'].value_counts(normalize=True)*100

In [ ]:
Current_Device_DF['non_reporting_flag'].value_counts(normalize=True)*100

In [ ]:
Current_Device_DF['device_type_desc'].value_counts(normalize=True)*100

In [ ]:
Current_Device_DF['device_class_desc'].value_counts(normalize=True)*100

In [ ]:
Current_Device_DF.groupby('customer_id')['device_class_desc'].count()

In [ ]:
Current_Device_DF['count_device']=1

In [ ]:
Device_class_DF=pd.pivot_table(Current_Device_DF, values='count_device', index='customer_id', columns='device_class_desc',
                          aggfunc='sum').reset_index()

In [ ]:
Device_class_DF=Device_class_DF.fillna(0)

In [ ]:
Device_class_DF.info()

In [ ]:
Device_class_DF.rename({'Keypad/Touchscreen': 'Keypad_Touchscreen'}, axis=1, inplace=True)

In [ ]:
Device_class_DF.columns = Device_class_DF.columns.str.replace(' ', '_')

In [ ]:
job_config_device_class_log = bigquery.job.LoadJobConfig()
job_config_device_class_log.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

table_Customer_device_class_count = 'ADC.customer_device_class_count'

job_ban_log = bq_client.load_table_from_dataframe(Device_class_DF, table_Customer_device_class_count,job_config=job_config_device_class_log)

In [ ]:
device_type_DF=pd.pivot_table(Current_Device_DF, values='count_device', index='customer_id', columns='device_type_desc',
                          aggfunc='sum')

In [ ]:
device_type_DF=device_type_DF.reset_index()

In [ ]:
device_type_DF.head()

In [ ]:
device_type_DF.info()

In [ ]:
rep_chars = ' |\|-|:|/'

device_type_DF.columns = device_type_DF.columns.str.replace(rep_chars, '_')

In [ ]:
device_type_DF.columns = device_type_DF.columns.str.replace('-', '_')

In [ ]:
device_type_DF.info()

In [ ]:
device_type_DF=device_type_DF.fillna(0)

In [ ]:
job_config_device_type_log = bigquery.job.LoadJobConfig()
job_config_device_type_log.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

table_Customer_device_type_count = 'ADC.customer_device_type_count'

job_device_type= bq_client.load_table_from_dataframe(device_type_DF, table_Customer_device_type_count,job_config=job_config_device_type_log)

In [ ]:
Camera_Event_schedule_sql= 'select * from `cio-datahub-enterprise-dv-e8ff.src_adc.bq_camera_event_schedules`'

In [ ]:
Camera_Event_schedule_DF= extract_bq_data(bq_client,sql=Camera_Event_schedule_sql)

In [ ]:
Camera_Event_schedule_DF.info()

In [ ]:
Camera_Event_schedule_DF.head()

In [ ]:
Camera_Event_schedule_DF['description'].value_counts(normalize=True)*100

In [ ]:
notification_schedule_sql='select customer_id,event_type_desc, count(*) as count_of_notification_schedules from `cio-datahub-enterprise-dv-e8ff.src_adc.bq_current_customer_notification_schedules` where enabled_status!=0 group by customer_id,event_type_desc order by customer_id'

In [ ]:
notification_schedule_DF= extract_bq_data(bq_client,sql=notification_schedule_sql)

In [ ]:
notification_schedule_DF.head()

In [ ]:
notification_schedule_DF['event_type_desc'].value_counts()

In [ ]:
job_config_notification_schedule = bigquery.job.LoadJobConfig()
job_config_notification_schedule.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

table_notification_schedule_count = 'ADC.customer_notification_schedule_count'

job_device_type= bq_client.load_table_from_dataframe(notification_schedule_DF, table_notification_schedule_count,job_config=job_config_notification_schedule)

In [ ]:
light_rules_sql= 'select * from cio-datahub-enterprise-dv-e8ff.src_adc.bq_light_rules'

In [ ]:
light_rules_DF= extract_bq_data(bq_client,sql=light_rules_sql)

In [ ]:
light_rules_DF.info()

In [ ]:
light_rules_DF.head()

In [ ]:
light_rules_DF['rule_type_desc'].value_counts()

In [ ]:
light_rules_DF['event_type_desc'].value_counts()